In [1]:
import warnings
warnings.filterwarnings('ignore')
from keras.datasets import mnist

Using TensorFlow backend.


In [2]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

11493376/11490434 [==============================] - 22s 2us/step


In [3]:
train_images.shape

(60000, 28, 28)

In [4]:
len(train_labels)

60000

In [5]:
train_labels

array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)

In [6]:
test_images.shape

(10000, 28, 28)

In [7]:
len(test_labels)

10000

In [8]:
test_labels

array([7, 2, 1, ..., 4, 5, 6], dtype=uint8)

In [9]:
from keras import models 
from keras import layers

In [10]:
network = models.Sequential()
network.add(layers.Dense(512, activation = 'relu', input_shape=(28 * 28,)))
network.add(layers.Dense(10, activation = 'softmax'))

In [11]:
network.compile(optimizer='rmsprop',
               loss = 'categorical_crossentropy',
               metrics=['accuracy'])

In [13]:
type(train_images)

numpy.ndarray

In [14]:
train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype('float32') / 255

In [15]:
from keras.utils import to_categorical

train_labels = to_categorical(train_labels)
test_labels = to_categorical(train_labels)

In [16]:
network.fit(train_images, train_labels, epochs = 5, batch_size = 128)

Epoch 1/5
60000/60000 [==============================] - 2s 35us/step - loss: 0.2618 - acc: 0.9239
Epoch 2/5
60000/60000 [==============================] - 1s 11us/step - loss: 0.1039 - acc: 0.9698
Epoch 3/5
60000/60000 [==============================] - 1s 11us/step - loss: 0.0685 - acc: 0.9790
Epoch 4/5
60000/60000 [==============================] - 1s 11us/step - loss: 0.0498 - acc: 0.9848
Epoch 5/5
60000/60000 [==============================] - 1s 11us/step - loss: 0.0372 - acc: 0.9884
